# A weighty task

- https://adventofcode.com/2023/day/14

For this task, you don't actually have to roll stones around. All you need to do is split each column on (consecutive sequences of) cubed rocks, and count the number of rolling rocks in each section. You can calculate their weight by taking the offset of the section into consideration.

If you started at the north end, the heaviest rock is weight 10, and if there are 4 rolling rocks you know that they all weigh more than $10 - 4 = 6$. Adding up consecutive numbers between to points (10 and 6 here) can be done by taking the [triangle number](https://en.wikipedia.org/wiki/Triangular_number) of the two values and subtracting them. Triangle numbers are trivial to compute, the formula is $\frac{n(n+1)}{2}$.

To split by consecutive cube-shaped rocks, split the string using a regular expression with a group in it; the [`re.split()` function](https://docs.python.org/3/library/re.html#re.split) (or the equivalent method on a compiled regular expression) then not only returns the strings between the pattern, but also the part that's matched by the group. Splitting on `(#+)` produces alternating strings with stationary cube-shaped boulders, and sections with rolling boulders and empty space. If the first character of a line were to be the north end of the map, then a boulder at that position would weigh `len(line)`, etc. As you process all the groups from a split, keep track of the maximum weight for that section by subtracting the length of each group as you iterate.

We do need to then re-orient our map to put the north-south line along text lines. We can use a simple Python transposition trick for this; if you pass a list of lines to the `zip()` function, as separate arguments, then this will yield tuples with the characters of each column. It's as if you rotated the input text by 90 degrees to the right then flipped the resulting lines.


In [1]:
import re
import typing as t


def _tn(n: int) -> int:
    """triangle number"""
    return n * (n + 1) // 2


def _by_column(map: str) -> list[str]:
    return ["".join(col) for col in zip(*map.splitlines())]


_cube_shaped = re.compile(r"(#+)")


def total_load(map: str) -> int:
    total = 0
    for col in _by_column(map):
        weight = len(col)
        for group in _cube_shaped.split(col):
            if rolling := group.count("O"):
                total += _tn(weight) - _tn(weight - rolling)
            weight -= len(group)
    return total


test_platform = """\
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
"""


assert total_load(test_platform) == 136

In [2]:
import aocd

platform = aocd.get_data(day=14, year=2023)
print("Part 1:", total_load(platform))

Part 1: 109098


# Cycling it up

Part two can be solved with more string manipulation. I did have to make two changes to my part 1 implementation for part 2:

- We can't just use transpositions now, we need proper rotations. Simply reverse each line after transposing from columns to rows.
- Calculating the weights needs to be a separate step now. I switched to just counting rolling rocks per line, and I reversed the map lines so the last line is processed first, etc. That allows us to use the [`enumerate()` function](https://docs.python.org/3/library/functions.html#enumerate) to provide us with the right weight value for each rolling rock.

Experienced participants will of course have recognized that we don't really want to cycle the map 1 billion times. Past AOC puzzles have taught us to look for repeating patterns: keep track of what the map looked like at each step and if you encounter the same map later on, you know how many steps have passed for this loop, and you can fast-forward to the end.


In [3]:
def _rotate(map: str) -> str:
    return "\n".join("".join(col[::-1]) for col in zip(*map.splitlines()))


def _roll(map: str) -> str:
    # move every rolling rock to the end of the group
    lines: list[str] = [
        "".join(
            [
                g.replace("O", "") + "O" * g.count("O")
                for g in t.cast(list[str], _cube_shaped.split(line))
            ]
        )
        for line in map.splitlines()
    ]
    return "\n".join(lines)


def _cycle(map: str) -> str:
    for _ in range(4):
        map = _roll(_rotate(map))
    return map


def cycle(map: str, steps: int) -> str:
    states: dict[str, int] = {}
    step = 0
    while step < steps:
        map = _cycle(map)
        if (prev := states.get(map)) is not None:
            # cycle found, we can fast-forward now
            length = step - prev
            step += (steps - step) // length * length
        else:
            states[map] = step
        step += 1
    return map


def total_load(map: str) -> int:
    return sum(
        row.count("O") * i for i, row in enumerate(reversed(map.splitlines()), 1)
    )


print(total_load(cycle(test_platform, 1_000_000_000)))

64


In [4]:
print(total_load(cycle(platform, 1_000_000_000)))

100064
